In [60]:
from __future__ import division, print_function
from keras.layers import Dense, Embedding, Flatten, Dropout, Convolution1D, MaxPooling1D
from keras.optimizers import Adam
from keras.models import Sequential
from keras.utils import get_file
from keras.preprocessing import sequence
import pickle
import numpy as np

In [5]:
# model_path = 'data/imdb/models/'
# %mkdir 


## Setup data

We're going to look at the IMDB dataset, which contains movies reviews from IMDB, along with their sentiment. Keras comes with some helpers for this dataset.

In [14]:
from keras.datasets import imdb
idx = imdb.get_word_index()

Using TensorFlow backend.


In [15]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

In [16]:
idx2word = {v: k for k, v in idx.items()}

I download the reviews using code copied from keras.datasets

In [24]:
path = get_file('imdb_full.pkl',
               origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [25]:
len(x_train)

25000

Here's the 1st review. As you see, the words have been replaced by ids. The ids can be looked up in idx2word.

In [26]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

The first word of the first review is 23022. Let's see what that is.

In [27]:
idx2word[23022]

'bromwell'

Here's the whole review, mapped from ids to words.

In [28]:
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

In [29]:
labels_train[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Reduce vocab size by setting rare words to max index

In [32]:
vocab_size = 5000

trn = [np.array([i if i < vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i <vocab_size-1 else vocab_size-1 for i in s ]) for s in x_test]

Look at distribution of lengths of sentences.

In [40]:
lens = np.array(map(len, trn))
(lens.max, lens.min, lens.mean)

(<function ndarray.max>, <function ndarray.min>, <function ndarray.mean>)

Pad (with zero) or truncate each sentence to make consistent length.

In [47]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

This results in nice rectangular matrics that can be passed to ML algorithms. Reviews shorter than 500 words are pre-padded with zeros, those greater are truncated.

In [48]:
trn.shape

(25000, 500)

## Create simple models

### Single hidden layer NN

The simplest model that tends to give reasonable results in a single hidden layer net. So let's try that. Note that we can't expect to get any useful results by feeding word ids directly into a neural net - so instead we use an embedding to replace them with a vector of 32 (initially random) floats for each word in the vocab.

In [54]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')
])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [56]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               1600100   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 1,760,201
Trainable params: 1,760,201
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.fit(trn, labels_train,validation_data=(test, labels_test), epochs=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 40s - loss: 0.4681 - acc: 0.7496 - val_loss: 0.2908 - val_acc: 0.8782
Epoch 2/2
25000/25000 [==============================] - 38s - loss: 0.2018 - acc: 0.9230 - val_loss: 0.3094 - val_acc: 0.8747


### Single conv layer with max pooling

A CNN is likely to work better, since it's designed to take advantage of ordered data. We'll need to use a 1D CNN, since a sequence of words is 1D.

In [61]:
conv1 = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, dropout=0.2),
    Dropout(0.2),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

D:\Miniconda\envs\model\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
D:\Miniconda\envs\model\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 5, activation="relu", padding="same")`
  after removing the cwd from sys.path.


In [62]:
conv1.compile(loss='binary_crossentropy', optimizer=Adam(),metrics=['accuracy'])

In [63]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), epochs=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 130s - loss: 0.4857 - acc: 0.7256 - val_loss: 0.2876 - val_acc: 0.8791
Epoch 2/4
25000/25000 [==============================] - 127s - loss: 0.2530 - acc: 0.9042 - val_loss: 0.2626 - val_acc: 0.8890
Epoch 3/4
25000/25000 [==============================] - 127s - loss: 0.2077 - acc: 0.9237 - val_loss: 0.2755 - val_acc: 0.8872
Epoch 4/4
25000/25000 [==============================] - 127s - loss: 0.1725 - acc: 0.9361 - val_loss: 0.2838 - val_acc: 0.8873


### Pre-trained vectors

In this section, we replicate the previous CNN, but using pre-trained embeddings.

In [ ]:
def get_glove_dataset(dataset):
    """Download the requested glove dataset from files.fast.ai
    and return a location that can be passed to load_vectors.
    """
    # generated from the original glove data

### Multi-size CNN

### LSTM

ModuleNotFoundError: No module named 'torch'